# One-to-Many and Many-to-Many Joins

## Introduction

Previously, you've learned about the typical case where one joins on a primary or foreign key. In this section, you'll explore other types of joins using One-to-Many and Many-to-many relationships!

## Objectives

You will be able to:

- Explain why Join Tables are needed in Many-to-Many relationships

## One-to-Many and Many-to-Many relationships

We've looked at a couple kinds of different join statements: left joins and inner joins. Both of these refer to the way in which we would like to define our joins based on the tables and their shared information. Another perspective on this is the number of matches between the tables based on our defined links with the keywords *on* or *using*.
  
We've investigated the typical case where one joins on a primary or foreign key. For example, when we join on customerID or employeeID, this value should be unique to that table. As such, our joins have been very similar to using a dictionary to find additional information associated with that record. In cases where there are multiple entries, in either table, for the field you are joining on, you will similarly be given multiple rows in your resulting view, one for each of these entries.  
  
For example, let's say you have another table 'restaurants' that has many columns including *name*, *city*, and *rating*. If you were to join this table with the offices table using the shared city column, you might get some unexpected behavior. That is, in the office table, there is only one office per city. However, because there is apt to be more then one restaurant for each of these cities in our second table, we will get unique combinations of Offices and Restaurants from our join. If there are 513 restaurants for Boston in our restaurant table and 1 office for Boston, our joined table will have each of these 513 rows, one for each restaurant along with the one office.

If we had 2 offices for Boston, and 513 restaurants, our join would have 1026 rows for boston; 513 for each restuarant along with the first office and 513 for each restaurant with the second office. Three offices in Boston would similarly produce 1539 rows; one for each unique combination of restaurants and offices. This is where you should be particularly careful of many to many joins as the resulting set size can explode drastically potentially consuming vast amounts of memory and other resources.  

<img src='Database-Schema.png' width=550>

## Connecting to the Database

In [1]:
import sqlite3
import pandas as pd

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
conn = sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_COLNAMES)
cur = conn.cursor()

## Checking Sizes of Resulting Joins...

### The original tables...

In [3]:
cur.execute('select * from offices;')
df = pd.DataFrame(cur.fetchall())
print('Number of results:', len(df))
df.head()

Number of results: 7


0              1                2                         3          4  \
0  1  San Francisco  +1 650 219 4782         100 Market Street  Suite 300   
1  2         Boston  +1 215 837 0825          1550 Court Place  Suite 102   
2  3            NYC  +1 212 555 3000      523 East 53rd Street    apt. 5A   
3  4          Paris  +33 14 723 4404  43 Rue Jouffroy D'abbans              
4  5          Tokyo  +81 33 224 5000               4-1 Kioicho              

            5       6         7      8  
0          CA     USA     94080     NA  
1          MA     USA     02107     NA  
2          NY     USA     10022     NA  
3              France     75017   EMEA  
4  Chiyoda-Ku   Japan  102-8578  Japan

In [4]:
cur.execute('select * from employees;')
df = pd.DataFrame(cur.fetchall())
print('Number of results:', len(df))
df.head()

Number of results: 23


0          1        2      3                                4  5     6  \
0  1002     Murphy    Diane  x5800     dmurphy@classicmodelcars.com  1         
1  1056  Patterson     Mary  x4611   mpatterso@classicmodelcars.com  1  1002   
2  1076   Firrelli     Jeff  x9273   jfirrelli@classicmodelcars.com  1  1002   
3  1088  Patterson  William  x4871  wpatterson@classicmodelcars.com  6  1056   
4  1102     Bondur   Gerard  x5408     gbondur@classicmodelcars.com  4  1056   

                      7  
0             President  
1              VP Sales  
2          VP Marketing  
3  Sales Manager (APAC)  
4   Sale Manager (EMEA)

### A One-to-One Join...

In [5]:
cur.execute('select * from offices join employees using(officeCode);')
df = pd.DataFrame(cur.fetchall())
print('Number of results:', len(df))
df.head()

Number of results: 23


0              1                2                  3          4   5    6   \
0  1  San Francisco  +1 650 219 4782  100 Market Street  Suite 300  CA  USA   
1  1  San Francisco  +1 650 219 4782  100 Market Street  Suite 300  CA  USA   
2  1  San Francisco  +1 650 219 4782  100 Market Street  Suite 300  CA  USA   
3  1  San Francisco  +1 650 219 4782  100 Market Street  Suite 300  CA  USA   
4  1  San Francisco  +1 650 219 4782  100 Market Street  Suite 300  CA  USA   

      7   8     9          10       11     12                              13  \
0  94080  NA  1002     Murphy    Diane  x5800    dmurphy@classicmodelcars.com   
1  94080  NA  1056  Patterson     Mary  x4611  mpatterso@classicmodelcars.com   
2  94080  NA  1076   Firrelli     Jeff  x9273  jfirrelli@classicmodelcars.com   
3  94080  NA  1143        Bow  Anthony  x5428       abow@classicmodelcars.com   
4  94080  NA  1165   Jennings   Leslie  x3291  ljennings@classicmodelcars.com   

     14                  15  
0                 President  
1  1002            VP Sales  
2  1002        VP Marketing  
3  1056  Sales Manager (NA)  
4  1143           Sales Rep

### A One-to-Many Join
Here we join products with product lines. There are only a few product lines that will be matched to each product. As a result, the product line descriptions will be repeated in our resulting view.

In [ ]:
cur.execute('select * from products;')
df = pd.DataFrame(cur.fetchall())
print('Number of results:', len(df))
df.head()

In [ ]:
cur.execute('select * from productlines;')
df = pd.DataFrame(cur.fetchall())
print('Number of results:', len(df))
df.head()

In [ ]:
cur.execute("""select * from products
                      join productlines
                      using(productLine);""")
df = pd.DataFrame(cur.fetchall())
print('Number of results:', len(df))
df.head()

### A Many-to-Many Join

If we join the employees and offices table, we will have a view with repeat cities listed.
(Recall this was 23 rows, one for each employee. Joining this with the customer table on the cities column will cause us to have a huge number of rows, one for each employee and customer combination for a given city.)

In [ ]:
cur.execute("""select * from employees
                        join offices
                        using(officeCode)
                        join customers
                        using(city);""")
df = pd.DataFrame(cur.fetchall())
print('Number of results:', len(df))
df.head()

In [ ]:
cur.execute("""select * from employees;""")
df = pd.DataFrame(cur.fetchall())
print('Number of results:', len(df))
df.head()

In [ ]:
cur.execute("""select * from customers;""")
df = pd.DataFrame(cur.fetchall())
print('Number of results:', len(df))
df.head()

## Summary

In this section, you expanded your Join knowledge to One-to-Many and Many-to-many Joins!